<a href="https://colab.research.google.com/github/jayteaftw/NLP-quora-duplicate-detection/blob/main/quora_duplicate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are only using test.csv(400k training examples). We are gonna do train, cv, test split on it. Lets just do a simple logistic regression on this and set a baseline performance for our model

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive/')
    path = '/content/gdrive/MyDrive/NLP_Final_Project/' 
except:
    path = ''

print(f"Path is {path}")


Path is 


In [2]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import OrderedDict
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_auc_score
from torch.utils.data import TensorDataset, DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cpu


# Init

In [80]:
path = ''
df = pd.read_csv(path+'train.csv')
y = np.array(df['is_duplicate'])
X = df[['id', 'qid1', 'qid2', 'question1', 'question2']]
X = np.array(X)
print(y.shape)
print(X.shape)
ones = np.count_nonzero(y == 1)
print(ones)
print(f"Ratio:\n 1: {(ones/len(y)) * 100}% \n 0: {(1-ones/len(y)) * 100}%")

(404290,)
(404290, 5)
149263
Ratio:
 1: 36.9197853026293% 
 0: 63.08021469737071%


In [81]:
X_train = df[['question1','question2' ]]
X_train = np.array(X_train)
X_train[:,0]

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       ..., 'What is one coin?',
       'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?',
       'What is like to have sex with cousin?'], dtype=object)

In [82]:
def train(train_dataset, test_dataset,feature_size, model, params={}, print_step=10):
    train_len = len(train_dataset)
    train_dataloader = DataLoader(train_dataset,batch_size=64)
    
    if test_dataset != None:
        test_len = len(test_dataset)
        test_dataloader = DataLoader(test_dataset,batch_size=64)

    

    loss_fn = params['loss']
    epochs = params['epochs']
    optimizer = params['optimizer']

    

    model.to(device)
    epoch_history = []
    fold_train_loss_history, fold_val_loss_history = [], []
    fold_val_acc_history = []
    
    for e in range(1,epochs+1):
        
        total_train_loss = 0
        total_train_correct = 0
        for batch_x, batch_y in train_dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    
            # Compute prediction error
            pred = torch.squeeze(model(batch_x))
            loss = loss_fn(pred, batch_y)
            
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #Save Training Loss and total correct
            total_train_loss += loss.item()
            out = (pred>0.5).float()
            total_train_correct += sum(out==batch_y).float().sum()
     
        if e == 1 or e % print_step == 0:
            with torch.no_grad():
                epoch_history.append(e)

                train_acc = total_train_correct / train_len
                avg_train_loss = total_train_loss / len(train_dataloader)
                fold_train_loss_history.append(avg_train_loss)
                if test_dataset != None:
                    total_val_correct = 0
                    total_val_loss = 0
                    for val_x, val_y in test_dataloader:
                        val_x, val_y = val_x.to(device), val_y.to(device)
                        val_pred = torch.squeeze(model(val_x))
                        val_out = (val_pred>0.5).float()
                        val_loss = loss_fn(val_pred, val_y)
                        total_val_loss += val_loss.cpu()
                        total_val_correct += sum(val_out==val_y).float().sum()

                    val_acc = total_val_correct / test_len
                    avg_val_loss = total_val_loss / len(test_dataloader)
                    fold_val_acc_history.append(val_acc.cpu())
                    fold_val_loss_history.append(avg_val_loss)
               

                    print(f"epoch: {e}, Train loss: {avg_train_loss:>4f},  Train Acc: {train_acc:>4f}, Val Loss: {avg_val_loss::>4f}, Val Acc: {val_acc:>4f}")
                else:
                    print(f"epoch: {e}, Train loss: {avg_train_loss:>4f},  Train Acc: {train_acc:>4f}")   
    return fold_train_loss_history, fold_val_loss_history, fold_val_acc_history, epoch_history,model

# Glove50 Logsitic Regregression Test


In [83]:
class Glove50Dataset():
    def __init__(self,X,y )-> None:
        

        word_embeddings = pd.read_csv(path+'glove.6B.50d.txt.zip',
                               header=None, sep=' ', index_col=0,
                               nrows=100000, compression='zip', encoding='utf-8', quoting=3)
        # Build a dict that will map from string word to 50-dim vector
        word_list = word_embeddings.index.values.tolist()
        word2vec = OrderedDict(zip(word_list, word_embeddings.values))
        

        q1 = []
        q2 = []
        a = []
        for (x1, x2),y1 in zip(X,y):
          if isinstance(x1, str) and isinstance(x2, str):
            q1.append(x1)
            q2.append(x2)
            a.append(y1)
          else:
            print(x1,x2)

        pair_len = len(q1)

        x_embed_q1 = np.zeros((pair_len, 50))
        x_embed_q2 = np.zeros((pair_len, 50))        
        for idx, (x1, x2) in enumerate(zip(q1,q2)):

          question1 = x1.strip().split(" ")
          question2 = x2.strip().split(" ")

          for word in question1:
            if word in word2vec:
              x_embed_q1[idx] += word2vec[word]
            x_embed_q1[idx] /= len(question1)

          for word in question2:
            if word in word2vec:
              x_embed_q2[idx] += word2vec[word]
            x_embed_q2[idx] /= len(question2)
        

        x_embed = np.concatenate((x_embed_q1,x_embed_q2),axis=1)
        print(x_embed.shape)

        self.feature_size = x_embed.shape[1]
        self.x_train = torch.Tensor(x_embed)[:100000]
        self.y_train = torch.Tensor(a)[:100000]
        print(self.feature_size, self.x_train.shape, self.y_train.shape)

    def __len__(self):
        return self.x_train.shape[0]
    
    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]


In [84]:
class LogisticModel(nn.Module):
    def __init__(self,input_size):
        super(LogisticModel,self).__init__()
        self.w = nn.Linear(input_size,1)

    def forward(self, x):
        return F.sigmoid(self.w(x))

In [85]:
dataset = Glove50Dataset(X_train,y)


How can I develop android app? nan
How can I create an Android app? nan
nan My Chinese name is Haichao Yu. What English name is most suitable for me considering the pronounciation of my Chinese name?
(404287, 100)
100 torch.Size([100000, 100]) torch.Size([100000])


In [ ]:
#For Testing
model = LogisticModel(dataset.feature_size) 

params = {  "loss": nn.BCELoss(),
            "optimizer":torch.optim.Adam(model.parameters(), lr=1e-2,betas=(0.9, 0.999)),
            "epochs": 60 }

kfold = KFold(n_splits=10, shuffle=True)
kfold.split(dataset)
train_loss_history, val_loss_history = [], []
val_acc_history = []
for fold, (train_idxs, test_idxs) in enumerate(kfold.split(dataset)):

    print(f'Starting fold {fold}')
    train_dataset = torch.utils.data.Subset(dataset,train_idxs)
    test_dataset = torch.utils.data.Subset(dataset,test_idxs)


    x_test = []
    y_test = []
    for idx in range(len(test_dataset)):
        input_data, target = test_dataset[idx]
        x_test.append(input_data)
        y_test.append(target)
    x_test = torch.stack(x_test)
    y_test = torch.tensor(y_test)


    fold_train_loss_history, fold_val_loss_history, \
    fold_val_acc_history, epoch_history, _ = train(\
                            train_dataset=train_dataset,
                            test_dataset=test_dataset,
                            feature_size=dataset.feature_size,
                            model=model,
                            params = params)
    

    pred = torch.squeeze(model(x_test))
    y_pred = (pred>0.5).float()
    print(f"F1 score: {f1_score(y_test, y_pred, pos_label=1, average='binary')}")
    print(f"AUC score: {roc_auc_score(y_test.detach().numpy(), model.forward(x_test).detach().numpy())}")

    
    train_loss_history.append(fold_train_loss_history)
    val_loss_history.append(fold_val_loss_history)
    val_acc_history.append(fold_val_acc_history) 

# Sentence Bert Model

In [3]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pickle
import os
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

/home/jaytea/Code_Workspace/Labs/Coen346/NLP-quora-duplicate-detection/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class SentenceTransformerDataset():
    def __init__(self,X,y, pair_len=0)-> None:
        
        model = SentenceTransformer('all-MiniLM-L6-v2')
        
        q1 = []
        q2 = []
        a = []
        for (x1, x2),y1 in zip(X,y):
          if isinstance(x1, str) and isinstance(x2, str):
            q1.append(x1)
            q2.append(x2)
            a.append(y1)
          else:
            print(x1,x2)

        pair_len = pair_len if pair_len else len(q1)

        q1, q2, a = q1[:pair_len], q2[:pair_len], a[:pair_len]
        x_embed_q1 = np.zeros((pair_len, 384))[:pair_len]
        x_embed_q2 = np.zeros((pair_len, 384)) [:pair_len]

        pool = model.start_multi_process_pool()


        #Start the multi-process pool on all available CUDA devices
        pool = model.start_multi_process_pool()

        step_size = 10000
        for start in tqdm(range(0, pair_len, step_size),total=pair_len//step_size):
          stop = start + step_size if (start + step_size) < pair_len else (pair_len - 1)
          x_embed_q1[start:stop] = model.encode_multi_process(q1[start:stop], pool)
          x_embed_q2[start:stop] = model.encode_multi_process(q2[start:stop], pool)

        #Optional: Stop the proccesses in the pool
        model.stop_multi_process_pool(pool)

        x_embed = np.concatenate((x_embed_q1,x_embed_q2),axis=1)
        
        print(x_embed.shape)

        self.feature_size = x_embed.shape[1]
        self.setences = [q1,q2]
        self.x_train = torch.Tensor(x_embed)
        self.y_train = torch.Tensor(a)
        print(self.feature_size, self.x_train.shape, self.y_train.shape)

    def __len__(self):
        return self.x_train.shape[0]
    
    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]



print()



In [7]:

create_bert_encoding = False
file_name = 'bert_encoding_max.pkl'


run = 'y'
if create_bert_encoding: 
  if os.path.isfile(path+file_name):
    run = input(f"'{file_name}' exists in the drive! Would you like to replace? (y/n)")
  if run == 'y':
    bert_dataset = SentenceTransformerDataset(X_train,y,pair_len=0)
    # Save the object to a file
    print(f'Writing Dataset to Drive')
    with open(path+file_name, 'wb') as f:
        pickle.dump(bert_dataset, f)
    print(f'Dataset Written!')

if not create_bert_encoding or run != 'y':
  # Load the object from the file
  if not os.path.isfile(path+file_name):
    raise ValueError("No Dataset in drive")
  else:
    print(f'Loading Dataset from Drive')
    with open(path+file_name, 'rb') as f:
        bert_dataset = pickle.load(f)
    print(f'Dataset Load!')


Loading Dataset from Drive
Dataset Load!


In [8]:
from torch.utils.data import random_split
train_percent = .8
val_percent = 1 - train_percent
sample_size = len(bert_dataset)
train_size = int(sample_size * train_percent)
val_size = sample_size - train_size


print(f'Train|Val Percent: {train_percent}|{val_percent}')
print(f'Train|Val Size: {train_size}|{val_size}')
# Split the data into train and test sets
train_dataset, test_dataset = random_split(bert_dataset, [train_size, val_size])

x_test = []
y_test = []
for idx in range(len(test_dataset)):
    input_data, target = test_dataset[idx]
    x_test.append(input_data)
    y_test.append(target)
x_test = torch.stack(x_test)
y_test = torch.tensor(y_test)


Train|Val Percent: 0.8|0.19999999999999996
Train|Val Size: 323429|80858


In [9]:
# Check data balance
indices = train_dataset.indices
targets = train_dataset.dataset.y_train

num_zeros = 0
num_ones = 0

for index in indices:
    label = targets[index]
    if label == 0:
        num_zeros += 1
    elif label == 1:
        num_ones += 1

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)
ratio = num_ones / (num_ones + num_zeros)
print(f"Training data Ratio:\n 1: {ratio * 100}% \n 0: {(1-ratio) * 100}%")

Number of 0s: 203972
Number of 1s: 119457
Training data Ratio:
 1: 36.93453586413092% 
 0: 63.06546413586908%


In [28]:
from sentence_transformers import util
import math

feature_size = bert_dataset.feature_size//2
total = 0
cos_scores = []
param = 0.8
for x, y in tqdm(test_dataset, total=len(test_dataset)):
  
  q1 = x[:feature_size]
  q2 = x[feature_size:]
  cosine = util.cos_sim(q1, q2).item()
  cos_scores.append(1 if cosine >=param else 0)
  if cosine >= param and y == 1 or cosine < param and y == 0:
    total += 1

percent = total / len(test_dataset)
print(f"Cosine Sim Accuracy: {percent}")
print(f"F1 score: {f1_score(y_test, cos_scores, average='binary')}")


100%|██████████| 80858/80858 [00:03<00:00, 23588.71it/s]

Cosine Sim Accuracy: 0.7818274011229563
F1 score: 0.7218692354990777


In [107]:
#For Training Simple Logisitc Regression with Bert

model = LogisticModel(bert_dataset.feature_size) 

params = {  "loss": nn.BCELoss(),
            "optimizer":torch.optim.Adam(model.parameters(), lr=1e-2,betas=(0.9, 0.999)),
            "epochs": 10 }


fold_train_loss_history, fold_val_loss_history, \
    fold_val_acc_history, epoch_history, _ = train(\
                            train_dataset=train_dataset,
                            test_dataset=test_dataset,
                            feature_size=bert_dataset.feature_size,
                            model=model,
                            params = params,
                            print_step=2)

pred = torch.squeeze(model(x_test))
y_pred = (pred>0.5).float()
print(f"F1 score: {f1_score(y_test, y_pred, pos_label=1, average='binary')}")
print(f"AUC score: {roc_auc_score(y_test.detach().numpy(), model.forward(x_test).detach().numpy())}")


epoch: 1, Train loss: 0.572727,  Train Acc: 0.708653, Val Loss: 0.567628, Val Acc: 0.713832
epoch: 2, Train loss: 0.566584,  Train Acc: 0.713000, Val Loss: 0.566740, Val Acc: 0.714215
epoch: 4, Train loss: 0.565799,  Train Acc: 0.713464, Val Loss: 0.566353, Val Acc: 0.714561
epoch: 6, Train loss: 0.565677,  Train Acc: 0.713585, Val Loss: 0.566254, Val Acc: 0.714957
epoch: 8, Train loss: 0.565646,  Train Acc: 0.713588, Val Loss: 0.566215, Val Acc: 0.715217
epoch: 10, Train loss: 0.565636,  Train Acc: 0.713498, Val Loss: 0.566196, Val Acc: 0.715093
F1 score: 0.5459705551942292
AUC score: 0.7478245615112822


In [111]:
print(bert_dataset.feature_size//2//2//2//2)
class LargerBertNetwork(nn.Module):
    def __init__(self,input_size):
        super(LargerBertNetwork,self).__init__()
        self.layer_1 = nn.Linear(input_size,384)
        self.layer_2 = nn.Linear(384,192)
        self.layer_3 = nn.Linear(192,96)
        self.layer_4 = nn.Linear(96,48)
        self.layer_f = nn.Linear(48,1)

    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = F.relu(self.layer_3(x))
        x = F.relu(self.layer_4(x))
        return F.sigmoid(self.layer_f(x))

48


In [112]:
#For Training Largers Model with Bert
model = LargerBertNetwork(bert_dataset.feature_size) 

params = {  "loss": nn.BCELoss(),
            "optimizer":torch.optim.Adam(model.parameters(), lr=1e-5,betas=(0.9, 0.999)),
            "epochs": 20 }


fold_train_loss_history, fold_val_loss_history, \
    fold_val_acc_history, epoch_history, _ = train(\
                            train_dataset=train_dataset,
                            test_dataset=test_dataset,
                            feature_size=bert_dataset.feature_size,
                            model=model,
                            params = params,
                            print_step=1)

pred = torch.squeeze(model(x_test))
y_pred = (pred>0.5).float()
print(f"F1 score: {f1_score(y_test, y_pred, pos_label=1, average='binary')}")
print(f"AUC score: {roc_auc_score(y_test.detach().numpy(), model.forward(x_test).detach().numpy())}")


epoch: 1, Train loss: 0.621545,  Train Acc: 0.649330, Val Loss: 0.579493, Val Acc: 0.713485
epoch: 2, Train loss: 0.553394,  Train Acc: 0.727424, Val Loss: 0.532930, Val Acc: 0.740335
epoch: 3, Train loss: 0.498440,  Train Acc: 0.761104, Val Loss: 0.465398, Val Acc: 0.777301
epoch: 4, Train loss: 0.423719,  Train Acc: 0.799471, Val Loss: 0.397122, Val Acc: 0.811682
epoch: 5, Train loss: 0.370333,  Train Acc: 0.828006, Val Loss: 0.365030, Val Acc: 0.827648
epoch: 6, Train loss: 0.343200,  Train Acc: 0.842939, Val Loss: 0.350163, Val Acc: 0.835848
epoch: 7, Train loss: 0.326063,  Train Acc: 0.852054, Val Loss: 0.341370, Val Acc: 0.841042
epoch: 8, Train loss: 0.312984,  Train Acc: 0.859366, Val Loss: 0.335446, Val Acc: 0.844715
epoch: 9, Train loss: 0.302110,  Train Acc: 0.865430, Val Loss: 0.331268, Val Acc: 0.847275
epoch: 10, Train loss: 0.292616,  Train Acc: 0.870488, Val Loss: 0.328285, Val Acc: 0.849440
epoch: 11, Train loss: 0.284088,  Train Acc: 0.875323, Val Loss: 0.326222, Val 